# Assignment 2
In this image classification implementation, we read Cifar10 dataset with pytorch, then build and train a CNN network model , and finally calculate the accuracy on test dataset.


In [3]:
# Libraries

import torch
from torch.utils.data import Dataset, Dataloader
import matplotlib.pyplot as plt
import torchvision


ModuleNotFoundError: No module named 'torch'